In [28]:
import getpass
import os
import requests
import json
import base64
from typing import List, TypedDict, Annotated, Optional
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage
from langgraph.graph.message import add_messages
from langgraph.graph import START, StateGraph
from langgraph.prebuilt import ToolNode, tools_condition
from IPython.display import Image, display
from langchain_core.tools import tool

# from igdb.wrapper import IGDBWrapper



import json

from dotenv import load_dotenv

load_dotenv()  # Loads from .env file

Client_ID= os.getenv("CLIENT_ID")
Client_Secret = os.getenv("CLIENT_SECRET")
gemini_key = os.getenv("GEMINI_API_KEY")

In [ ]:
import requests
import json

def get_igdb_access_token(client_id: str, client_secret: str) -> str:
    """
    Obtains an access token from the Twitch OAuth2 endpoint for IGDB API access.
    """
    url = "https://id.twitch.tv/oauth2/token"
    params = {
        'client_id': client_id,
        'client_secret': client_secret,
        'grant_type': 'client_credentials'
    }
    
    try:
        response = requests.post(url, data=params)
        response.raise_for_status() 
        token_info = response.json()
        print("Successfully retrieved token information from Twitch:")
        print(json.dumps(token_info, indent=4)) # This will print the full token response
        return token_info['access_token']

    except requests.exceptions.HTTPError as err_h:
        print(f"HTTP Error obtaining token: {err_h}")
        print(f"Response content: {response.text}")
        return None
    except requests.exceptions.RequestException as err:
        print(f"An error occurred while obtaining token: {err}")
        return None
    except json.JSONDecodeError:
        print("Error: Could not decode JSON response while obtaining token.")
        print(f"Raw response content: {response.text}")
        return None
    except KeyError as err_k:
        print(f"Error: Missing key in token response - {err_k}")
        print(f"Response content: {token_info}")
        return None

# --- EXECUTE THE CALL IN THIS CELL ---

# IMPORTANT: Replace these with your actual Twitch Client ID and Client Secret
# (These are the ones you used for the token endpoint)
my_client_id = "51ykn9y98ltfxwt8lkqdoogr3npz5w" 
my_client_secret = "2utcabv5oemodshhj0ox61alw179xp"

igdb_access_token = get_igdb_access_token(my_client_id, my_client_secret)

if igdb_access_token:
    print(f"\nFinal Access Token obtained (truncated): {igdb_access_token[:10]}...")
else:
    print("\nAccess token not obtained. Please check your client ID and secret.")


--- Running Cell 1: Obtaining Access Token ---
Successfully retrieved token information from Twitch:
{
    "access_token": "gtzankh97a26ilbolqncifp9sm59qg",
    "expires_in": 5290282,
    "token_type": "bearer"
}

Final Access Token obtained (truncated): gtzankh97a...


In [30]:
import requests
import json # Ensure json is imported in this cell too for independence

def get_igdb_characters(client_id: str, access_token: str) -> list:

    url = "https://api.igdb.com/v4/characters"
    
    headers = {
        "Client-ID": client_id,
        "Authorization": f"Bearer {access_token}",
        "Accept": "application/json" 
    }
    
    # The body is a plain text string for IGDB's "query language"
    body = "fields id, name, gender, species, description; limit 10;"

    try:
        response = requests.post(url, headers=headers, data=body)
        response.raise_for_status() 
        
        characters_data = response.json()
        print("\nSuccessfully retrieved IGDB character information:")
        print(json.dumps(characters_data, indent=4)) # This will print the character data
        
        return characters_data

    except requests.exceptions.HTTPError as err_h:
        print(f"HTTP Error fetching characters: {err_h}")
        print(f"Response content: {response.text}")
        return None
    except requests.exceptions.ConnectionError as err_c:
        print(f"Error Connecting to IGDB API: {err_c}")
        return None
    except requests.exceptions.Timeout as err_t:
        print(f"Timeout Error fetching characters: {err_t}")
        return None
    except requests.exceptions.RequestException as err:
        print(f"An unexpected error occurred while fetching characters: {err}")
        return None
    except json.JSONDecodeError:
        print("Error: Could not decode JSON response from IGDB characters endpoint.")
        print(f"Raw response content: {response.text}")
        return None

# --- EXECUTE THE CALL IN THIS CELL ---

# Ensure that 'igdb_access_token' and 'my_client_id' are available from Cell 1.
# If you run this cell without running Cell 1 first, these variables might not exist.
if 'igdb_access_token' in locals() and igdb_access_token:
    print("\n--- Running Cell 2: Fetching IGDB Characters ---")
    characters_list = get_igdb_characters(my_client_id, igdb_access_token)
    
    if characters_list:
        print(f"\nTotal characters retrieved (final confirmation): {len(characters_list)}")
        # You can access specific data like:
        # for char in characters_list:
        #     print(f"  - Character Name: {char.get('name')}")
    else:
        print("\nFailed to retrieve IGDB characters. Check the above error messages.")
else:
    print("\nAccess token not available. Please run Cell 1 first to obtain the token.")


--- Running Cell 2: Fetching IGDB Characters ---

Successfully retrieved IGDB character information:
[
    {
        "id": 8303,
        "gender": 0,
        "name": "Raven",
        "species": 5
    },
    {
        "id": 12822,
        "description": "Peach (or Princess Peach) is a main protagonist in the Mario series. She's usually the damsel-in-distress of the games, who the players have to rescue, but she also has some important roles as the hero herself. She's the adorable and cute princess of the Mushroom Kingdom.",
        "gender": 1,
        "name": "Peach",
        "species": 1
    },
    {
        "id": 3305,
        "description": "Jolyne Cujoh is the protagonist of Part 6 and the sixth JoJo of the JoJo's Bizarre Adventure series. \n \nJolyne is a young woman of above-average height and slim to athletic build. \n \nBold-eyed, she wears her hair in two \"layers\": A dark base including two large buns atop her head and a short length going down her neck; above which, lightl

In [34]:


def search_game_by_name(client_id: str, access_token: str, game_name: str):
    """
    Searches for a game by name on IGDB and returns its ID and basic info.
    Prioritizes the most relevant result (limit 1).
    """
    url = "https://api.igdb.com/v4/games"
    headers = {
        "Client-ID": client_id,
        "Authorization": f"Bearer {access_token}",
        "Accept": "application/json"
    }
    # Request game ID, name, summary, and URL.
    # The 'search' command is used for fuzzy matching on game names.
    body = f'search "{game_name}"; fields id, name, summary, url; limit 1;'

    try:
        response = requests.post(url, headers=headers, data=body)
        response.raise_for_status() # Raises an HTTPError for bad responses (4xx or 5xx)
        game_data = response.json()
        
        if game_data:
            print(f"\n--- Found Game: {game_data[0]['name']} (ID: {game_data[0]['id']}) ---")
            if game_data[0].get('summary'):
                print(f"Summary: {game_data[0]['summary'][:150]}... (truncated)" if len(game_data[0]['summary']) > 150 else game_data[0]['summary'])
            return game_data[0] # Return the first matching game dictionary
        else:
            print(f"\nNo game found matching '{game_name}'.")
            return None
    except requests.exceptions.RequestException as e:
        print(f"\nError searching for game '{game_name}': {e}")
        if 'response' in locals() and response:
            print(f"Response content: {response.text}")
        return None
    except json.JSONDecodeError:
        print(f"\nError decoding JSON response for game search. Raw response: {response.text}")
        return None

def get_characters_for_game(client_id: str, access_token: str, game_id: int):
    """
    Retrieves detailed character information associated with a specific game ID from IGDB.
    """
    url = "https://api.igdb.com/v4/characters"
    headers = {
        "Client-ID": client_id,
        "Authorization": f"Bearer {access_token}",
        "Accept": "application/json"
    }
    # Request character fields: name, description, gender, species, and their URL.
    # 'where games = ({game_id})' filters characters linked to the specific game ID.
    # Max limit for characters is 500 per request; adjust or paginate if more are needed.
    body = f'fields name, description, gender, species, url; where games = ({game_id}); limit 500;'

    try:
        response = requests.post(url, headers=headers, data=body)
        response.raise_for_status() # Raises an HTTPError for bad responses (4xx or 5xx)
        characters_data = response.json()

        if characters_data:
            print(f"\n--- Found {len(characters_data)} characters for this game ---")
            return characters_data
        else:
            print(f"\nNo characters found for game ID {game_id}.")
            return [] # Return an empty list if no characters
    except requests.exceptions.RequestException as e:
        print(f"\nError fetching characters for game ID {game_id}: {e}")
        if 'response' in locals() and response:
            print(f"Response content: {response.text}")
        return None
    except json.JSONDecodeError:
        print(f"\nError decoding JSON response for characters. Raw response: {response.text}")
        return None

# --- Main execution block for this cell ---

print("\n--- Running Cell: Search Game and Get Characters ---")

# Verify that client_id and access_token are available from the previous cell.
# If this cell is run independently without the token acquisition cell,
# these variables won't exist, leading to an error.
if 'my_client_id' not in locals() or 'igdb_access_token' not in locals() or not igdb_access_token:
    print("Error: 'my_client_id' or 'igdb_access_token' not found.")
    print("Please ensure you have run the 'Get IGDB Access Token' cell successfully first.")
else:
    # Prompt user for game name
    game_to_search = input("Enter the name of the game you want to search for (e.g., 'The Witcher 3', 'Cyberpunk 2077'): ")
    
    # 1. Search for the game
    found_game_info = search_game_by_name(my_client_id, igdb_access_token, game_to_search)

    if found_game_info:
        game_id = found_game_info['id']
        game_name = found_game_info['name']
        
        # 2. Get characters for the found game ID
        characters_in_game = get_characters_for_game(my_client_id, igdb_access_token, game_id)

        if characters_in_game:
            print(f"\nDetailed Character Information for '{game_name}':")
            for char in characters_in_game:
                print("-" * 30)
                print(f"Name: {char.get('name', 'N/A')}")
                print(f"Gender: {char.get('gender', 'N/A')}")
                print(f"Species: {char.get('species', 'N/A')}")
                
                description = char.get('description')
                if description:
                    # Truncate long descriptions for readability
                    print(f"Description: {description[:200]}... (truncated)" if len(description) > 200 else description)
                
                char_url = char.get('url')
                if char_url:
                    print(f"More info: {char_url}")
            print("-" * 30)
        elif characters_in_game == []: # Explicitly check for empty list
            print(f"\nNo characters found for '{game_name}' in the IGDB API.")
        else: # Handle cases where get_characters_for_game returned None due to an error
            print(f"\nFailed to retrieve characters for '{game_name}' due to an error.")
    else:
        print(f"\nGame '{game_to_search}' was not found or an error occurred during search.")


--- Running Cell: Search Game and Get Characters ---

--- Found Game: God of War (ID: 19560) ---
Summary: God of War is the sequel to God of War III as well as a continuation of the canon God of War chronology. Unlike previous installments, this game focus... (truncated)

--- Found 13 characters for this game ---

Detailed Character Information for 'God of War':
------------------------------
Name: Athena
Gender: 1
Species: 5
More info: https://www.igdb.com/characters/athena
------------------------------
Name: Kratos
Gender: 0
Species: 5
Description: Kratos was born in Sparta and became a fierce warrior. In a desperate moment during battle, he pledged his life to Ares in exchange for victory, becoming his servant. Ares, manipulating Kratos, caused... (truncated)
More info: https://www.igdb.com/characters/kratos
------------------------------
Name: Atreus
Gender: 0
Species: 5
More info: https://www.igdb.com/characters/atreus
------------------------------
Name: Mimir
Gender: 0
Species

In [ ]:
class AgentState(TypedDict):
    # The document provided
    messages: Annotated[list[AnyMessage], add_messages]

In [ ]:
def _set_env(key: str):
    if key not in os.environ:
        os.environ[key] = getpass.getpass(f"{key}:")

In [ ]:
@tool
def get_game_characters(game_name: str) -> List[dict]:
    """
    Returns a list of characters for a given video game using the IGDB API.

    Parameters:
        game_name: The name of the game.

    Returns:
        A list of dictionaries containing 'id' and 'name' keys.
    """
    response = wrapper.api_request(
        'games',
        f'search "{game_name}"; fields id, name;'
    )
    games = json.loads(response.decode('utf-8'))

    if not games:
        raise ValueError(f"No game found for name: {game_name}")

    game_id = games[0]['id']

    char_response = wrapper.api_request(
        'characters',
        f'fields id, name, games; where games = {game_id};'
    )
    characters = json.loads(char_response.decode('utf-8'))

    return [{'id': c['id'], 'name': c['name']} for c in characters]


In [ ]:
# manually checking the function get_game_characters
characters = get_game_characters("god of war")
for char in characters:
    print(f"{char['id']}: {char['name']}")

In [ ]:
# Equip the butler with tools
tools = [
    get_game_characters
]

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
llm_with_tools = llm.bind_tools(tools)

In [ ]:
def assistant(state: AgentState):
    # Tool description
    textual_description_of_tool = """
get_game_characters(game_name: str) -> list:
    Returns a list of characters for a given video game using the IGDB API.

    Args:
        game_name: The name of the game (e.g., "God of War", "Valorant").

    Returns:
        A list of dictionaries where each contains 'id' and 'name' of a character.
"""

    # image = state["input_file"]
    sys_msg = SystemMessage(
        content=(
            f"You are a helpful assistant that helps users learn about video games, their characters, and related information. "
            f"You can fetch video game characters using this tool:\n"
            f"{textual_description_of_tool}\n"
        )
    )

    return {
        "messages": [llm_with_tools.invoke([sys_msg] + state["messages"])],
        # "input_file": state["input_file"]
    }

In [ ]:
# The graph
builder = StateGraph(AgentState)

builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    tools_condition,
)
builder.add_edge("tools", "assistant")
react_graph = builder.compile()

# Show the butler's thought process
display(Image(react_graph.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
messages = [HumanMessage(content="Who are the characters in God of War?")]

try:
    response = react_graph.invoke({"messages": messages})
    for m in response['messages']:
        print(m.content)
except Exception as e:
    print("Error:", e)